# Import continuous data from .continuous Openephys raw files

### Ideas
<ul>
    <li>Crear una clase Mouse donde poner todo por ratón?</li>
    <li>Definitivamente crear una clase con los filtros</li>
    </ul>

In [ ]:
""" The file works from terminal taking the name of the mouse as its only argument.
E.g. Run:

python3 continuous_epochs.py 'SERT1597'

to preprocess the mouse 'SERT1957'


Output: 
    1. Info files
    2. Matrix by structure, with 
        - Channels at 30 KHz
        - All channels with mean subtracted
        - Low-passed at 300 Hz
        - Median subtracted
        - Saved at SERTXXXX/npys_dir + structure.npy
        
""" 

#### Import the required modules

In [1]:
# Import required modules
import sys
sys.path.insert(1, '/home/diogenes/Projects/SERT/modules/')

import glob
import numpy as np
import pandas as pd
import physig as ps
from scipy import signal
from matplotlib import pyplot as plt

#### Get the mouse name and create the file name and folder paths

In [2]:
if len(sys.argv) > 1:
    ID = sys.argv[1]
else:
    print('Bad file format!')
    exit()


channels_list = ['ch01', 'ch02', 'ch03', 'ch04', 'ch05', 'ch06', 'ch07', 'ch08', 'ch09', 'ch10',
                 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch20',
                 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch30',
                 'ch31', 'ch32']

print('Processing mouse {}...'.format(ID))

# Setting working file and paths
files_dir = '/home/diogenes/Projects/SERT/DATA/MICE/' + ID + '/continuous/'
npys_dir  = '/home/diogenes/Projects/SERT/DATA/MICE/' + ID + '/npys/'
figs_dir  = '/home/diogenes/Projects/SERT/DATA/MICE/' + ID + '/figs/'

# List all continuous files in working folder
files = sorted(glob.glob(files_dir + '*.continuous'))

# Read the list of channels
df = pd.read_csv(files_dir + 'canales.csv', header=None, names=["locs"])
channels_locations = np.array(df['locs'].tolist())
n_channels = len(channels_locations)
print(channels_locations)

# Start and end of OF
df = pd.read_csv(files_dir + 'xy.csv', header=None)
start_OF = int(np.ceil(df[3][0] * 30)) # Times 30 because of sampling at 30 KHz
stop_OF = int(np.floor(df[3][1] * 30))

# Read the entrances times
df = pd.read_csv(files_dir + 'entradas.csv', header=None, names=["locs"])
entrances_times = np.array(df['locs'].tolist(), dtype='int') * 30 # Times 30 because of sampling at 30 KHz

n_epochs = len(entrances_times)

print("Entrances times: {}".format(entrances_times))
print('Number of entrances: {}'.format(n_epochs))

# Collect the indexes for each structure
mPFC_indexes  = [i for i,x in enumerate(channels_locations) if x == 'mPFC_left']
NAC_indexes = [i for i,x in enumerate(channels_locations) if x == 'NAC_left']
BLA_indexes  = [i for i,x in enumerate(channels_locations) if x == 'BLA_left']
vHip_indexes  = [i for i,x in enumerate(channels_locations) if x == 'vHipp_left']

# Get the number of channels for each structure
mPFC_nchannels = len(mPFC_indexes)
NAC_nchannels  = len(NAC_indexes)
BLA_nchannels  = len(BLA_indexes)
vHip_nchannels = len(vHip_indexes)

# Create and save info file
info = {'channels_list': channels_list, 'channels_locs': channels_locations, 'n_channels': n_channels,
        'mPFC_nchannels': mPFC_nchannels, 'NAC_nchannels': NAC_nchannels, 'BLA_nchannels': BLA_nchannels,
        'vHip_nchannels': vHip_nchannels, 'entrances_times': entrances_times, 'n_epochs': n_epochs,
        'startOF': start_OF, 'stopOF': stop_OF}

np.save(npys_dir + 'info.npy', info)

print(info)

print('Info file: saved\n')

Processing mouse 1597...
['mPFC_left' 'mPFC_left' 'mPFC_left' 'mPFC_left' 'mPFC_left' 'mPFC_left'
 'mPFC_left' 'mPFC_left' 'mPFC_left' 'mPFC_left' 'NAC_left' 'NAC_left'
 'NAC_left' 'NAC_left' 'NAC_left' 'NAC_left' 'BLA_left' 'BLA_left'
 'BLA_left' 'BLA_left' 'BLA_left' 'BLA_left' 'BLA_left' 'BLA_left'
 'BLA_left' 'vHipp_left' 'vHipp_left' 'vHipp_left' 'vHipp_left'
 'vHipp_left' 'vHipp_left' 'vHipp_left']
Entrances times: [20315070 20709060 21989070 22590060 23822070 24127080 25368060 26129070
 27057060 28125060 29705070 29923080 30330060 30958080 31049070 31734060]
Number of entrances: 16
{'channels_list': ['ch01', 'ch02', 'ch03', 'ch04', 'ch05', 'ch06', 'ch07', 'ch08', 'ch09', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch30', 'ch31', 'ch32'], 'channels_locs': array(['mPFC_left', 'mPFC_left', 'mPFC_left', 'mPFC_left', 'mPFC_left',
       'mPFC_left', 'mPFC_left', 'mPFC_